<a href="https://colab.research.google.com/github/dnevo/Practicum---project-6/blob/master/Practicum_project_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# following lines are specific to Google Colab
#%load_ext google.colab.data_table
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks/Practicum Yandex - Project 6'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Practicum Yandex - Project 6


In [0]:
import pandas as pd
import numpy as np
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:11,.2f}'.format

In [3]:
companies_trips = pd.read_csv('datasets/users_behavior.csv')
companies_trips.head()

,Unnamed: 0,calls,minutes,messages,mb_used,is_ultra
0,0,40.00,311.90,83.00,"19,915.42",0
1,1,85.00,516.75,56.00,"22,696.96",0
2,2,77.00,467.66,86.00,"21,060.45",0
3,3,106.00,745.53,81.00,"8,437.39",1
4,4,66.00,418.74,1.00,"14,502.75",0
